In [ ]:
%matplotlib inline
import pandas as pd 
import datetime
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.layers import (Concatenate, Dense, LSTM, Input, Dropout, LeakyReLU, 
                                     TimeDistributed, RepeatVector)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import matplotlib.pyplot as plt
import numpy as np
from Processing import Holiday


In [ ]:
train = df["load"][:-8760]
test = df["load"][-8760:]
mean, std = train.mean(), train.std()
def data_generator(data, backward, forward, mean, std):
    data = data.values
    N = len(data)
    normalized = ((data - mean)/std)
    targets = TimeseriesGenerator(data, data, length=forward, batch_size=N)
    target, _ = targets[0]
    series = TimeseriesGenerator(normalized[:-forward], target, length=backward, batch_size=N-forward)
    input, output = series[0]
    return input, output
  
input_train, output_train = data_generator(train, 168, 24, mean, std)
input_test, output_test = data_generator(test, 168, 24, mean, std)
n, m = input_train.shape 
input_train = input_train.reshape(n, m, 1)
dropout = 0.1

In [ ]:
input_layer = Input(shape=(m, 1))
seq = LSTM(units=200, dropout=dropout)(input_layer)
seq = RepeatVector(24)(seq)
seq = LSTM(units=200, dropout=dropout, return_sequences=True)(seq)
seq = TimeDistributed(Dense(units=100, activation=tf.nn.relu))(seq)
output_layer = TimeDistributed(Dropout(dropout))(seq)
output_layer = TimeDistributed(Dense(1))(output_layer)

model = Model(inputs=input_layer, outputs=output_layer)
optimizer = Adam()

model.compile(
        loss="mean_squared_error",
        optimizer=optimizer,
        metrics=["mean_absolute_error", "mean_squared_error"])

In [ ]:
checkpoint_name = 'checkpoints/Weights.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, 
                            monitor='val_loss',
                            verbose = 1, 
                            save_best_only = True, 
                            mode ='min')

In [ ]:
history = model.fit(input_train,
                    output_train,
                    epochs=50,
                    validation_split=0.3,
                    verbose=1,
                    callbacks=[checkpoint])

In [ ]:
model.load_weights("checkpoints/Weights.hdf5")
pred = model.predict(input_test)

In [ ]:
def multi_step_plot(history, true_future, prediction):
  plt.figure(figsize=(12, 6))
  num_in = len(history)
  num_out = len(true_future)

  plt.plot(np.arange(num_in), history, label='History')
  plt.plot(np.arange(num_in, num_in + num_out),true_future, 'bo',
           label='True Future')
  if prediction.any():
    plt.plot(np.arange(num_in, num_in + num_out), prediction, 'ro',
             label='Predicted Future')
  plt.legend(loc='upper left')
  plt.show()

  multi_step_plot(input_test[0]*std+mean, output_test[0], pred[0])